In [1]:
# importing required libraries
import numpy as np
import cv2
from collections import deque

In [2]:
# Declaring lower and upper limit of BGR values of blue color
blueLower = np.array([100, 60, 60])
blueUpper = np.array([140, 255, 255])

# Define a 5x5 kernel for erosion and dilation
kernel = np.ones((5, 5), np.uint8)

# Initialize deques to store different colors in different arrays
bpoints = [deque(maxlen=512)]
gpoints = [deque(maxlen=512)]
rpoints = [deque(maxlen=512)]
ypoints = [deque(maxlen=512)]

# Initialize an index variable for each of the colors 
bindex = 0
gindex = 0
rindex = 0
yindex = 0

# Just a handy array and an index variable to get the color-of-interest on the go
# Blue, Green, Red, Yellow respectively
colors = [(255, 0, 0), (0, 255, 0), (0, 0, 255), (0, 255, 255)] 
colorIndex = 0

In [3]:
# creating a blank white interface
paintingWin = np.zeros((471,636,3)) + 255    # here 255 is the color code for white

# draw color selection rectangular buttons on the created white interface (img,startCoord,endCoord,color,thickness)
paintingWin = cv2.rectangle(paintingWin, (40,1), (140,65), (0,0,0), 2)      # clear button
paintingWin = cv2.rectangle(paintingWin, (160,1), (200,65), colors[0], -1)  # blue
paintingWin = cv2.rectangle(paintingWin, (275,1), (370,65), colors[1], -1)  # green
paintingWin = cv2.rectangle(paintingWin, (390,1), (485,65), colors[2], -1)  # red
paintingWin = cv2.rectangle(paintingWin, (505,1), (600,65), colors[3], -1)  # yellow

# labeling the respective boxes (img,text,org,font,fontscale,color,thickness,linetype)
cv2.putText(paintingWin, "CLEAR ALL", (49, 33), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 2, cv2.LINE_AA)
cv2.putText(paintingWin, "BLUE", (185, 33), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA)
cv2.putText(paintingWin, "GREEN", (298, 33), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA)
cv2.putText(paintingWin, "RED", (420, 33), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA)
cv2.putText(paintingWin, "YELLOW", (520, 33), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (150,150,150), 2, cv2.LINE_AA)

array([[[255., 255., 255.],
        [255., 255., 255.],
        [255., 255., 255.],
        ...,
        [255., 255., 255.],
        [255., 255., 255.],
        [255., 255., 255.]],

       [[255., 255., 255.],
        [255., 255., 255.],
        [255., 255., 255.],
        ...,
        [255., 255., 255.],
        [255., 255., 255.],
        [255., 255., 255.]],

       [[255., 255., 255.],
        [255., 255., 255.],
        [255., 255., 255.],
        ...,
        [255., 255., 255.],
        [255., 255., 255.],
        [255., 255., 255.]],

       ...,

       [[255., 255., 255.],
        [255., 255., 255.],
        [255., 255., 255.],
        ...,
        [255., 255., 255.],
        [255., 255., 255.],
        [255., 255., 255.]],

       [[255., 255., 255.],
        [255., 255., 255.],
        [255., 255., 255.],
        ...,
        [255., 255., 255.],
        [255., 255., 255.],
        [255., 255., 255.]],

       [[255., 255., 255.],
        [255., 255., 255.],
        [255., 2

In [4]:
# start capturing the live video
vid = cv2.VideoCapture(0)

# checking the status of VideoCapture
if vid.isOpened():
    vid = cv2.VideoCapture(0)
    print("yes!")
if not vid.isOpened():
    raise IOError("Cannot open webcam!")

yes!


In [5]:
# creating a window to show output painting
cv2.namedWindow("Paint")  # naming the created window

while (True):
    
    # PREPARING THE CANVAS FOR EVERY LIVE FRAMES 

    # grab each frames while reading the video and converting to HSV scale
    ret, frame = vid.read()
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)   # Q) Why conversion to HSV?
    
    # add the coloring options to the frame
    frame = cv2.rectangle(frame, (40,1), (140,65), (122,122,122), -1)
    frame = cv2.rectangle(frame, (160,1), (255,65), colors[0], -1)
    frame = cv2.rectangle(frame, (275,1), (370,65), colors[1], -1)
    frame = cv2.rectangle(frame, (390,1), (485,65), colors[2], -1)
    frame = cv2.rectangle(frame, (505,1), (600,65), colors[3], -1)
    
    # add the labels to rectangular boxes in each live frame for color access
    cv2.putText(frame, "CLEAR ALL", (49, 33), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA)
    cv2.putText(frame, "BLUE", (185, 33), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA)
    cv2.putText(frame, "GREEN", (298, 33), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA)
    cv2.putText(frame, "RED", (420, 33), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA)
    cv2.putText(frame, "YELLOW", (520, 33), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (150,150,150), 2, cv2.LINE_AA)
    

    # LOCATING THE TARGETED BLUE COLORED BOTTLE CAP

    # Determine which pixels fall within the blue boundaries and then blur the binary image
    blueMask = cv2.inRange(gray, blueLower, blueUpper)
    blueMask = cv2.erode(blueMask, kernel, iterations=2)
    blueMask = cv2.morphologyEx(blueMask, cv2.MORPH_OPEN, kernel)
    blueMask = cv2.dilate(blueMask, kernel, iterations=1)

    
    # Find contours in the image (img,contour retreival mode,contour approximation)
    (cnts, _) = cv2.findContours(blueMask.copy(), cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
        
    # check to see if any contours (blue stuff) were found
    if len(cnts) > 0:
            # sort the contours and find the largest one -- we assume this contour correspondes to the area of the bottle/pen cap
            cnt = sorted(cnts, key = cv2.contourArea, reverse = True)[0]
            # get the radius of the enclosing circle around the found contour
            ((x, y), radius) = cv2.minEnclosingCircle(cnt)
            # draw the circle around the found contour
            cv2.circle(frame, (int(x), int(y)), int(radius), (0, 255, 255), 2)
            # Moments to calculate the centre of the contours(ie.e bottle cap)
            M = cv2.moments(cnt)
            center = (int(M['m10'] / M['m00']), int(M['m01'] / M['m00']))
         
        
            if center[1] <= 65:  # if y corrdiante of the centre is less than 65(ie in range of canvas containing the buttons
                if 40 <= center[0] <= 140: # Clear All Button
                    
                    # Giving different arrays to handle colour points of different colour. These arrays 
                    # will hold the points of a particular colour in the array which will further be used to draw on canvas
                    bpoints = [deque(maxlen=512)]
                    gpoints = [deque(maxlen=512)]
                    rpoints = [deque(maxlen=512)]
                    ypoints = [deque(maxlen=512)]

                    # indexes to mark position of pointers in colour array
                    bindex = 0
                    gindex = 0
                    rindex = 0
                    yindex = 0

                    # setting diifferent value of colorIndex for different colors
                    paintingWin[67:,:,:] = 255
                elif 160 <= center[0] <= 255:
                        colorIndex = 0 # Blue
                elif 275 <= center[0] <= 370:
                        colorIndex = 1 # Green
                elif 390 <= center[0] <= 485:
                        colorIndex = 2 # Red
                elif 505 <= center[0] <= 600:
                        colorIndex = 3 # Yellow
            
            else :  # if the centre is not in range of 0 to 65 pixel then draw lines of the selected color
                if colorIndex == 0:
                    bpoints[bindex].appendleft(center)
                elif colorIndex == 1:
                    gpoints[gindex].appendleft(center)
                elif colorIndex == 2:
                    rpoints[rindex].appendleft(center)
                elif colorIndex == 3:
                    ypoints[yindex].appendleft(center)
        
        # Append the next deque when no contours are detected (i.e., bottle cap reversed)
    else:
        bpoints.append(deque(maxlen=512))
        bindex += 1
        gpoints.append(deque(maxlen=512))
        gindex += 1
        rpoints.append(deque(maxlen=512))
        rindex += 1
        ypoints.append(deque(maxlen=512))   
        yindex += 1

        
    # DISPLAYING THE DRAWINGS ON SCREEN
    #  storing the colors deques
    points = [bpoints, gpoints, rpoints, ypoints]
    
    # drawing the lines of different colors
    for i in range(len(points)):
        for j in range(len(points[i])):
            for k in range(1, len(points[i][j])):
                if points[i][j][k - 1] is None or points[i][j][k] is None:
                    continue
                cv2.line(frame, points[i][j][k - 1], points[i][j][k], colors[i], 2) #(img,star_point,end_point,color,thickness)
                cv2.line(paintingWin, points[i][j][k - 1], points[i][j][k], colors[i], 2)

    # show the frame and the paintWindow image
    cv2.imshow("Tracking", frame)
    cv2.imshow("Paint", paintingWin)

    # If the 'q' key is pressed, stop the loop
    if cv2.waitKey(1) & 0xFF == ord("q"):
        break
        
vid.release()
cv2.destroyAllWindows()        